Just need to update the transition probabilities / functionals; everything else should carry over from the Contact Process case. We should work on Z so that we have a state space of $R^3.$ Otherwise it will be $R^{2n+1}$

# Defining Measures 
We want to work with measures that are constant on dyadic intervals.

In [17]:
import numpy as np
import itertools as itr
def create_k_measure(vector,k):
    
    side = 2**k
    # use numpy array to model this measure
    measure = np.array(vector).reshape((side,side,side))
    
    # have to normalize this measure
    return measure/measure.sum()
    
    
def create_random_k_measure(k):
    return create_k_measure(np.random.uniform(0,1,2**(3*k)),k)

def smoothen(measure,k):
    side_length = measure.shape[0]
    assert side_length >= 2**k
    
    if side_length == 2**k:
        return measure
    
    smoothed_measure = np.zeros(shape = (2**k,2**k,2**k))
    smoothed_idxs = np.arange(0,2**k)
    smoothed_length = int(side_length/(2**k))
    for i,j,k in itr.product(smoothed_idxs,smoothed_idxs,smoothed_idxs):
        smoothed_measure[i,j,k] = measure[(smoothed_length*i):(smoothed_length*(i+1)),
                                          (smoothed_length*j):(smoothed_length*(j+1)),
                                          (smoothed_length*k):(smoothed_length*(k+1))].sum()
        
    
    return smoothed_measure

# Defining dynamics for Parallel Ising
## To do: vectorize logic, and modularize all the code, plot convergence

We have the same setup as before; we jsut need to change the functionals: For $\mathbf{x} = (\mathbf{x}_{-1},\mathbf{x}_0,\mathbf{x}_1) \in [0,1]^3$ we can write the transition kernel of the nonlinear Markov process as
$$
\Prob^\mu(\mathbf{x},\cdot) = \operatorname{Law}(\frac{1}{2}\mathbf{x} + \frac{1}{2}\mathbf{v}(\mathbf{x},\mu)),
$$
where 
$$
\mathbf{v}(\mathbf{x},\mu) \sim \Ber(F_{(-1)}(\mu,\mathbf{x})) \otimes \Ber(F_{(0)}(\mu,\mathbf{x})) \otimes \Ber(F_{(1)}(\mu,\mathbf{x}))
$$
is a product measure, with the functionals $F_v(\mu,\mathbf{x}), v\in V$ defined as follows:
\begin{align*}
    F_{(0)}(\mu,\mathbf{x}) & = p\frac{\exp(\beta(\floor{2\mathbf{x}_{1}} + \floor{2\mathbf{x}_{-1}})}{\exp(\beta(\floor{2\mathbf{x}_{1}} + \floor{2\mathbf{x}_{-1}})) + \exp(-\beta(\floor{2\mathbf{x}_{1}} + \floor{2\mathbf{x}_{-1}}))} \\
    F_{(1)}(\mu,\mathbf{x}) & = \E^{\mathbf{y} \sim \mu}\left[ p\frac{\exp(\beta(\floor{2\mathbf{y}_{1}} + \floor{2\mathbf{y}_{-1}})}{\exp(\beta(\floor{2\mathbf{xy}_{1}} + \floor{2\mathbf{y}_{-1}})) + \exp(-\beta(\floor{2\mathbf{y}_{1}} + \floor{2\mathbf{y}_{-1}}))}  \ | \ \mathbf{y}_0 = \mathbf{x}_1, \mathbf{y}_{-1} = \mathbf{x}_0 \right] \\
    F_{(-1)}(\mu,\mathbf{x}) & = \E^{\mathbf{y} \sim \mu}\left[ p\frac{\exp(\beta(\floor{2\mathbf{y}_{1}} + \floor{2\mathbf{y}_{-1}})}{\exp(\beta(\floor{2\mathbf{y}_{1}} + \floor{2\mathbf{y}_{-1}})) + \exp(-\beta(\floor{2\mathbf{y}_{1}} + \floor{2\mathbf{y}_{-1}}))}  \ | \ \mathbf{y}_0 = \mathbf{x}_{-1}, \mathbf{y}_{1} = \mathbf{x}_0 \right] \\
\end{align*}

The functionals compute the probability each node evolves to the value $1.$ The above formulas are only valid when the current state is -1, and it flips to 1. Take the difference from 1 of the above probabilities, if the current state is 1, and multiply the inside of the gibbs probability by -1.

## should just recpakage by writing function to compute flip probability

In [18]:
# sign is the sign of the ending flipped state
# computes probability under gibbs measure; helper function
def conditioned_gibbs_measure(beta,x,y, sign):
    prob = np.exp(sign*beta*(x+y))
    psum = prob + 1/prob
    return prob/psum

# We can define the current state with respect to the measure itself, by specifying the indices.
def compute_conditional_expectation_ising(measure, case, current_state, beta,p):
    side_length = measure.shape[0]
    expectation = 0
    
    get_first_crd = lambda x: int(2*x/side_length)
    
    # get the current sign of the state
    current_sign = 0
    
    if case == 1:
        conditioned_measure = measure[current_state[1],current_state[2],:]
        current_sign = get_first_crd(current_state[2])
        
    elif case == -1:
        conditioned_measure = measure[:,current_state[0],current_state[1]]
        current_sign = get_first_crd(current_state[0])
    else: 
        print('case should be either 1,-1.')
    
    # renormalize the conditioned measure
    if conditioned_measure.sum() != 0:
            conditioned_measure = conditioned_measure/conditioned_measure.sum()
      
    
    # can definitely vectorize this logic
    floor_y1 = get_first_crd(current_state[1])
    
    # If current sign is -1
    if current_sign == 0:
        # create function by plugging in what is known
        gibbs_functional = lambda idx: conditioned_gibbs_measure(beta, 2*floor_y1-1, 2*get_first_crd(idx)-1,sign = 1)
    
        # compute the expectation by looping through indices
        expectation = p*sum([prob*gibbs_functional(idx) for idx, prob in enumerate(conditioned_measure)])
        
    # If current sign is +1
    else:
        # create function by plugging in what is known
        gibbs_functional = lambda idx: conditioned_gibbs_measure(beta, 2*floor_y1-1, 2*get_first_crd(idx)-1, sign = -1)
    
        # compute the expectation by looping through indices
        expectation = 1 - p*sum([prob*gibbs_functional(idx) for idx, prob in enumerate(conditioned_measure)])
    return expectation
    
    
    
# computes the functionals above.  
def compute_functionals_ising(measure, coordinate, current_state, beta,p):
    side_length = measure.shape[0]
    midpoint_idx = int(side_length/2)
    functional = 0
    
    if coordinate == 0:
        get_first_crd = lambda x: int(2*x/side_length)
        # if sign is -1
        if get_first_crd(current_state[1]) == 0:
            functional = p*conditioned_gibbs_measure(beta,2*get_first_crd(current_state[0])-1,
                                                 2*get_first_crd(current_state[2])-1,sign = 1)
        # if sign is 1
        else:
            functional = 1 - p*conditioned_gibbs_measure(beta,2*get_first_crd(current_state[0])-1,
                                                 2*get_first_crd(current_state[2])-1, sign = -1)
            
    elif coordinate == -1:
        functional = compute_conditional_expectation_ising(measure,-1,current_state,beta,p)
            
    elif coordinate == 1:
        functional = compute_conditional_expectation_ising(measure,1,current_state,beta,p)
        
    else:
        print("coordinate should be one of -1,0,1")
        
    return functional

# computes transition probability 
# transition map is a 3-tuple in {0,1}^3, which describes how to transition to the next state
# it is the vector v above
def compute_transition_probability_ising(measure, current_state, transition_map,beta,p):
    
    probs = [0]*3
    for i in range(3):
        probs[i] = compute_functionals_ising(measure, i-1, current_state, beta,p)
    
    probs = np.array(probs)
    m = np.array([1-probs,probs])
    
    return np.product(m[transition_map,range(3)])

In [19]:
# returns another measure, of the same discretization
def transition_ising(measure,beta,p):
    side_length = measure.shape[0]
    new_side_length = 2*side_length
    new_measure = np.zeros([new_side_length]*3)
    
    # transition for each element in the old measure
    for i,j,k in itr.product(range(side_length),range(side_length),range(side_length)):
        for transition in list(itr.product([0,1],[0,1],[0,1])):
            
            transition_prob = compute_transition_probability_ising(measure, [i,j,k], transition, beta,p)
            new_coordinates = np.array([i,j,k]) + side_length*np.array(transition)
            new_measure[tuple(new_coordinates)] = \
                new_measure[tuple(new_coordinates)] + \
                measure[i,j,k]*transition_prob 
    
    return smoothen(new_measure, int(np.log2(side_length)))

def simulate_nonlinear_dynamics_ising(initial_measure,beta,p):
    mu_0 = initial_measure
    mu_current = mu_0

    consec_diff = []
    while True:
        mu_next = transition_ising(mu_current,beta,p)
        consec_diff = consec_diff + [np.sqrt(((mu_next - mu_current)**2).sum())] 
        mu_current = mu_next
        if consec_diff[-1] < 1e-10:
            break

    return (consec_diff, mu_current)

def simulate_nonlinear_dynamics_for_fixed_steps_ising(initial_measure,beta,p,steps):
    mu_0 = initial_measure
    mu_current = mu_0

    consec_diff = []
    for i in range(steps):
        mu_next = transition_ising(mu_current,beta,p)
        consec_diff = consec_diff + [np.sqrt(((mu_next - mu_current)**2).sum())] 
        mu_current = mu_next

    return (consec_diff, mu_current)

def l2norm(x,y):
    return np.sqrt(((x-y)**2).sum())

## Low Temperature Regime
We took $\beta = 10$ and $p = 0.9.$ We tend to see polarization.

In [43]:
np.random.seed(10)
measure = create_random_k_measure(k = 1)
print(measure)

[[[ 0.2000094   0.00538114]
  [ 0.16430988  0.19417063]]

 [[ 0.12926672  0.05829151]
  [ 0.05135923  0.19721149]]]


In [44]:
for cds in itr.product((0,1),repeat = 3):
    print("%s,%.4f"%(cds,compute_functionals_ising(measure,-1,cds,10,0.9)))

(0, 0, 0),0.1767
(0, 0, 1),0.1767
(0, 1, 0),0.8620
(0, 1, 1),0.8620
(1, 0, 0),0.2072
(1, 0, 1),0.2072
(1, 1, 0),0.7767
(1, 1, 1),0.7767


In [45]:
for cds in itr.product((0,1),repeat = 3):
    print("%s,%.4f"%(cds,compute_functionals_ising(measure,1,cds,10,0.9)))

(0, 0, 0),0.0118
(0, 0, 1),0.3437
(0, 1, 0),0.5899
(0, 1, 1),0.9070
(1, 0, 0),0.0118
(1, 0, 1),0.3437
(1, 1, 0),0.5899
(1, 1, 1),0.9070


In [33]:
compute_functionals_ising(measure,0,[0,0,0],10,0.9)

3.8235188297624304e-18

In [48]:
results = simulate_nonlinear_dynamics_for_fixed_steps_ising(measure,10,0.9,steps=500)
print(results[1]) # get the stationary measure

[[[ 0.48256494  0.00245882]
  [ 0.00267929  0.00245984]]

 [[ 0.00246466  0.00268032]
  [ 0.00246569  0.50222644]]]


Check if the smoothened stationary distribution for greater k are the stationary distributions for smaller k

In [7]:
measure = create_random_k_measure(k = 2)
print(measure)

[[[ 0.00476282  0.02649513  0.01671016  0.01639072]
  [ 0.01559598  0.02665595  0.02232754  0.02308394]
  [ 0.02202233  0.00910561  0.00879803  0.02001458]
  [ 0.00784889  0.0126662   0.0087994   0.02156025]]

 [[ 0.00360988  0.009702    0.02721205  0.02029806]
  [ 0.02635264  0.01267375  0.00566255  0.02448698]
  [ 0.0033472   0.01318009  0.00328022  0.01508716]
  [ 0.00780406  0.01531278  0.01705262  0.02104334]]

 [[ 0.02206396  0.02586966  0.01732339  0.00202432]
  [ 0.01379478  0.02759431  0.01036804  0.02757847]
  [ 0.00239937  0.01578935  0.02772978  0.02674343]
  [ 0.01964023  0.01393712  0.01463862  0.01199744]]

 [[ 0.01460492  0.01418061  0.00513019  0.02416929]
  [ 0.0216053   0.01643604  0.01569115  0.02367435]
  [ 0.01565094  0.01584836  0.0083146   0.0007185 ]
  [ 0.00369134  0.01914681  0.00668196  0.02602049]]]


In [16]:
results = simulate_nonlinear_dynamics_for_fixed_steps_ising(measure,10,0.9,steps=1500)
print(results[0])
print(smoothen(results[1],k=1)) # get the stationary measure

[0.055534131186395136, 0.098343413253718051, 0.055318098397167108, 0.039939857732055259, 0.032277134210247443, 0.027424329218061282, 0.023936065551221433, 0.02122177566382169, 0.019022847786219873, 0.017192662597526623, 0.015643475530218397, 0.014314867609244877, 0.013164287438303951, 0.012159439423083711, 0.011275648277383364, 0.010493290813139287, 0.0097967459815059051, 0.0091732933973941808, 0.0086125631944482459, 0.0081059783673734925, 0.0076464263269588094, 0.0072279473250646343, 0.0068455287293125869, 0.0064949203708609586, 0.0061725028951600727, 0.0058751737905127438, 0.0056002617418933476, 0.0053454539555953867, 0.0051087394753425172, 0.0048883615014011193, 0.0046827791768819594, 0.0044906354908168661, 0.0043107310218003488, 0.0041420018210078114, 0.0039835010290650442, 0.0038343833095443912, 0.0036938917437894184, 0.0035613466627392931, 0.0034361361446122044, 0.0033177078623464742, 0.0032055620845308267, 0.003099245630360362, 0.0029983466393445849, 0.0029024900250714451, 0.002

We see that the point masses at zero and 1, i.e. all the states are -1 or +1, are fixed points of the process.

In [228]:
vec = np.zeros(8**1)
vec[-1] = 1
point_mass_at_one = create_k_measure(vec,1)

vec = np.zeros(8**1)
vec[0] = 1
point_mass_at_zero = create_k_measure(vec,1)
point_mass_at_zero

array([[[ 1.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.]]])

In [233]:
simulate_nonlinear_dynamics_for_fixed_steps_ising(point_mass_at_one,10,0.9,50)

([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 array([[[ 0.,  0.],
         [ 0.,  0.]],
 
        [[ 0.,  0.],
         [ 0.,  1.]]]))

In [232]:
simulate_nonlinear_dynamics_for_fixed_steps_ising(point_mass_at_zero,10,0.9,50)

([6.6225288768448271e-18,
  5.1420506481217398e-18,
  4.3782594030920445e-18,
  3.891440091554671e-18,
  3.5511074227782447e-18,
  3.3021458540896332e-18,
  3.1155414393903255e-18,
  2.9737457822999181e-18,
  2.8651849057861225e-18,
  2.7817941683693128e-18,
  2.7177473061477013e-18,
  2.6687290412971827e-18,
  2.6314852873401409e-18,
  2.6035286825825144e-18,
  2.5829375127722857e-18,
  2.5682140290471147e-18,
  2.5581822296902623e-18,
  2.5519127663214149e-18,
  2.548666988720625e-18,
  2.5478547717230313e-18,
  2.5490024271788628e-18,
  2.5517280909884793e-18,
  2.5557227092614609e-18,
  2.5607352557382304e-18,
  2.5665611713669061e-18,
  2.573033274324184e-18,
  2.5800145758538128e-18,
  2.5873925747393372e-18,
  2.5950747051127427e-18,
  2.6029846884149319e-18,
  2.6110595975758787e-18,
  2.6192474848132248e-18,
  2.627505457428715e-18,
  2.6357981112183895e-18,
  2.6440962505203231e-18,
  2.6523758389201953e-18,
  2.6606171362757669e-18,
  2.6688039867985934e-18,
  2.676923230038

We should expect to see that initial states with high probability of having all $-1,+1$ should have stationary measures that are these fixed points?

In [262]:
# add some noise to get a initial measure close to the point mass
nearpm1 = point_mass_at_one + np.abs(np.random.normal(0,0.1,(2,2,2))) 
nearpm1 = nearpm1/nearpm1.sum()
print(nearpm0)

nearpm0 = point_mass_at_zero + np.abs(np.random.normal(0,0.1,(2,2,2)))
nearpm0 = nearpm0/nearpm0.sum()
print(nearpm1)

[[[  7.08938394e-01   1.44791650e-01]
  [  3.89225226e-04   3.06854598e-02]]

 [[  2.29926064e-02   1.88721817e-02]
  [  1.16573073e-02   6.16731755e-02]]]
[[[ 0.05606331  0.0257335 ]
  [ 0.04108867  0.03677087]]

 [[ 0.04786727  0.05011374]
  [ 0.10403181  0.63833084]]]


In [268]:
results1 = simulate_nonlinear_dynamics_for_fixed_steps_ising(nearpm1,10,0.9,2000)
print(results1[1])
results0 = simulate_nonlinear_dynamics_for_fixed_steps_ising(nearpm0,10,0.9,2000)
print(results0[1])

[[[  1.15763343e-01   2.62657981e-04]
  [  2.84627356e-04   2.63761284e-04]]

 [[  2.62888768e-04   2.85731132e-04]
  [  2.63993018e-04   8.82612998e-01]]]
[[[  7.51121037e-01   4.76186598e-04]
  [  5.15559689e-04   4.74875305e-04]]

 [[  4.76018275e-04   5.14248622e-04]
  [  4.74707435e-04   2.45947367e-01]]]


In [267]:
# add some noise to get a initial measure close to the point mass
nearpm1 = point_mass_at_one + np.abs(np.random.normal(0,0.1,(2,2,2))) 
nearpm1 = nearpm1/nearpm1.sum()

nearpm0 = point_mass_at_zero + np.abs(np.random.normal(0,0.1,(2,2,2)))
nearpm0 = nearpm0/nearpm0.sum()

results1 = simulate_nonlinear_dynamics_for_fixed_steps_ising(nearpm1,10,0.9,2000)
print(results1[1])
results0 = simulate_nonlinear_dynamics_for_fixed_steps_ising(nearpm0,10,0.9,2000)
print(results0[1])

[[[  1.15763343e-01   2.62657981e-04]
  [  2.84627356e-04   2.63761284e-04]]

 [[  2.62888768e-04   2.85731132e-04]
  [  2.63993018e-04   8.82612998e-01]]]
[[[  7.51121037e-01   4.76186598e-04]
  [  5.15559689e-04   4.74875305e-04]]

 [[  4.76018275e-04   5.14248622e-04]
  [  4.74707435e-04   2.45947367e-01]]]


There seems to be two stationary distributions, depending on the weight put on $+1$ versus $-1$ states.

## High Temperature Regime
Take $\beta = 0.1.$ We expect to see well mixed things.

In [247]:
measure = create_random_k_measure(k = 1)
print(measure)

[[[ 0.29084654  0.00709503]
  [ 0.07755741  0.21913112]]

 [[ 0.0181472   0.17834235]
  [ 0.06828806  0.14059228]]]


In [252]:
results = simulate_nonlinear_dynamics_for_fixed_steps_ising(measure,0.1,0.9,1000)
print(results[1]) # get the stationary measure, seems to be unique

[[[ 0.1306903   0.12394478]
  [ 0.12142014  0.12394478]]

 [[ 0.12394478  0.12142014]
  [ 0.12394478  0.1306903 ]]]


In [13]:
measure = create_random_k_measure(k = 2)
print(measure)
results = simulate_nonlinear_dynamics_for_fixed_steps_ising(measure,0.1,0.9,500)
print(smoothen(results[1],1)) # get the stationary measure, seems to be unique

[[[ 0.02042138  0.01969937  0.01062424  0.0177168 ]
  [ 0.01220859  0.02974742  0.00216908  0.02228255]
  [ 0.00732176  0.00874614  0.00355853  0.02405882]
  [ 0.01510043  0.02965532  0.00344168  0.02127468]]

 [[ 0.01480562  0.01281216  0.01670961  0.01418181]
  [ 0.02906867  0.02479983  0.00990968  0.0094982 ]
  [ 0.02336678  0.0046153   0.02118701  0.02233604]
  [ 0.00429387  0.0064347   0.02256217  0.01546869]]

 [[ 0.02023044  0.00720754  0.00149134  0.02779306]
  [ 0.00838482  0.02525588  0.01882424  0.01146508]
  [ 0.02971297  0.00213316  0.00947755  0.02454306]
  [ 0.02911937  0.01797343  0.02480154  0.00606869]]

 [[ 0.02503772  0.02371388  0.02718546  0.02090209]
  [ 0.00248491  0.01620355  0.00172123  0.01002399]
  [ 0.00058215  0.02899032  0.01131165  0.00115835]
  [ 0.00497712  0.01570965  0.02163831  0.02380051]]]
[[[ 0.1306903   0.12394478]
  [ 0.12142014  0.12394478]]

 [[ 0.12394478  0.12142014]
  [ 0.12394478  0.1306903 ]]]


In [15]:
measure = create_random_k_measure(k = 3)
print(measure)
results = simulate_nonlinear_dynamics_for_fixed_steps_ising(measure,0.1,0.9,500)
print(smoothen(results[1],1)) # get the stationary measure, seems to be unique

[[[  2.44761925e-03   2.56948999e-03   2.99953701e-03   3.42204236e-03
     2.25517018e-03   2.04833036e-03   2.58248986e-03   2.90030519e-03]
  [  1.59792703e-03   8.68717533e-04   5.02327725e-04   2.62132405e-03
     2.67660956e-03   3.09223014e-03   1.90550946e-03   1.12939448e-03]
  [  3.07541362e-03   1.38738768e-03   2.40168130e-03   2.62307311e-03
     2.28516664e-03   3.94294188e-04   1.69870614e-03   3.10014738e-03]
  [  2.43390311e-04   3.25607351e-03   1.30209839e-03   2.85308567e-03
     2.38502778e-04   3.12639034e-03   2.77118472e-03   2.63210903e-04]
  [  2.43287768e-03   1.35266908e-03   1.74046565e-03   3.45309009e-03
     1.09795098e-04   2.15610939e-03   3.27104641e-04   3.62626246e-03]
  [  2.15754229e-03   3.52920328e-03   1.67773448e-03   2.25730537e-04
     6.57472275e-04   4.59461915e-04   3.42993024e-03   3.28507489e-03]
  [  1.52870401e-03   3.23389820e-03   8.51684897e-04   2.55216544e-03
     2.13814913e-03   2.74022538e-03   3.15906518e-03   1.28980177e-03]

[[[ 0.1306903   0.12394478]
  [ 0.12142014  0.12394478]]

 [[ 0.12394478  0.12142014]
  [ 0.12394478  0.1306903 ]]]


What if the temperature is infinite? Then the stationary distribution is exactly uniform.

In [254]:
measure = create_random_k_measure(k = 1)
print(measure)

results = simulate_nonlinear_dynamics_for_fixed_steps_ising(measure,0,0.9,1000)
print(results[1]) # get the stationary measure


[[[ 0.18259478  0.08891292]
  [ 0.18558583  0.12275297]]

 [[ 0.11287171  0.12128481]
  [ 0.04418137  0.14181559]]]
[[[ 0.125  0.125]
  [ 0.125  0.125]]

 [[ 0.125  0.125]
  [ 0.125  0.125]]]



$\newcommand{\Prob}{\mathbb{P}}
\DeclareMathOperator{\Law}{Law}
\DeclareMathOperator{\Ber}{Bernoulli}
\newcommand{\set}[1]{\lbrace #1\rbrace}
\newcommand{\ceil}[1]{\left\lceil #1 \right\rceil}
\newcommand{\floor}[1]{\left\lfloor #1 \right\rfloor}
\newcommand{\E}{\mathbb{E}}$

# Defining dynamics for the Parallel Voter Process
## Might need to redefine so as to have stationary distributions
We have the same setup as before; we jsut need to change the functionals: For $\mathbf{x} = (\mathbf{x}_{-1},\mathbf{x}_0,\mathbf{x}_1) \in [0,1]^3$ we can write the transition kernel of the nonlinear Markov process as
$$
\Prob^\mu(\mathbf{x},\cdot) = \operatorname{Law}(\frac{1}{2}\mathbf{x} + \frac{1}{2}\mathbf{v}(\mathbf{x},\mu)),
$$
where 
$$
\mathbf{v}(\mathbf{x},\mu) \sim \Ber(F_{(-1)}(\mu,\mathbf{x})) \otimes \Ber(F_{(0)}(\mu,\mathbf{x})) \otimes \Ber(F_{(1)}(\mu,\mathbf{x}))
$$
is a product measure, with the functionals $F_v(\mu,\mathbf{x}), v\in V$ defined as follows:
\begin{align*}
    F_{(0)}(\mu,\mathbf{x}) & = \mathbf{1}_{\set{\floor{2\mathbf{x}_{-1}} + \floor{2\mathbf{x}_1} > 1}} + \frac{1}{2}\mathbf{1}_{\set{\floor{2\mathbf{x}_{-1}} + \floor{2\mathbf{x}_1} = 1}} \\
    F_{(1)}(\mu,\mathbf{x}) & = \Prob^{\mathbf{y} \sim \mu}[ \floor{2\mathbf{y}_{-1}} + \floor{2\mathbf{y}_1} > 1 \ | \ \mathbf{y}_0 = \mathbf{x}_1, \mathbf{y}_{-1} = \mathbf{x}_0]) + \frac{1}{2}\Prob^{\mathbf{y} \sim \mu}[ \floor{2\mathbf{y}_{-1}} + \floor{2\mathbf{y}_1} = 1 \ | \ \mathbf{y}_0 = \mathbf{x}_1, \mathbf{y}_{-1} = \mathbf{x}_0]) \\
    F_{(-1)}(\mu,\mathbf{x}) & = \Prob^{\mathbf{y} \sim \mu}[ \floor{2\mathbf{y}_{-1}} + \floor{2\mathbf{y}_1} > 1 \ | \ \mathbf{y}_0 = \mathbf{x}_{-1}, \mathbf{y}_{1} = \mathbf{x}_0]) + \frac{1}{2}\Prob^{\mathbf{y} \sim \mu}[ \floor{2\mathbf{y}_{-1}} + \floor{2\mathbf{y}_1} = 1 \ | \ \mathbf{y}_0 = \mathbf{x}_{-1}, \mathbf{y}_{1} = \mathbf{x}_0]) 
\end{align*}

The functionals compute the probability each node evolves to the value $1.$

In [70]:
# We can define the current state with respect to the measure itself, by specifying the indices.
def compute_conditional_expectation_voter(measure, case, current_state):
    side_length = measure.shape[0]
    expectation = 0
    if case == 1:
        conditioned_measure = measure[current_state[1],current_state[2],:]
    elif case == -1:
        conditioned_measure = measure[:,current_state[0],current_state[1]]
    else: 
        print('case should be either 1,-1.')
    
    # renormalize the conditioned measure
    if conditioned_measure.sum() != 0:
            conditioned_measure = conditioned_measure/conditioned_measure.sum()
            
    # 1- floor(2*y_0)
    sum_thresh = 1 - int(2*current_state[1]/side_length)
    
    # compute the prob that neighbors vote > 1
    majority_prob = sum([prob for idx,prob in enumerate(conditioned_measure) if idx >= (sum_thresh+1)*side_length/2 ])

    # compute the prob that neighbors vote == 1
    tie_probability = sum([prob for idx,prob in enumerate(conditioned_measure) if idx >= sum_thresh*side_length/2 \
                           and idx < (sum_thresh+1)*side_length/2])
    
    
    return majority_prob + 0.5*tie_probability
    
# computes the functionals above.  
def compute_functionals_voter(measure, coordinate, current_state):
    side_length = measure.shape[0]
    midpoint_idx = int(side_length/2)
    functional = 0
    
    if coordinate == 0:
        neighbor_votes = (int(current_state[0] < midpoint_idx) + int(current_state[2] < midpoint_idx))
        functional = (neighbor_votes == 2) + 0.5*(neighbor_votes == 1)
            
    elif coordinate == -1:
        functional = compute_conditional_expectation_voter(measure,-1,current_state)
            
    elif coordinate == 1:
        functional = compute_conditional_expectation_voter(measure,1,current_state)
        
    else:
        print("coordinate should be one of -1,0,1")
        
    return functional

# computes transition probability 
# transition map is a 3-tuple in {0,1}^3, which describes how to transition to the next state
# it is the vector v above
def compute_transition_probability_voter(measure, current_state, transition_map):
    
    probs = [0]*3
    for i in range(3):
        probs[i] = compute_functionals_voter(measure, i-1, current_state)
    
    probs = np.array(probs)
    m = np.array([1-probs,probs])
    
    return np.product(m[transition_map,range(3)])


In [217]:
import matplotlib.pyplot as plt
import pandas as pd

# returns another measure, of the same discretization
def transition_voter(measure):
    side_length = measure.shape[0]
    new_side_length = 2*side_length
    new_measure = np.zeros([new_side_length]*3)
    
    # transition for each element in the old measure
    for i,j,k in itr.product(range(side_length),range(side_length),range(side_length)):
        for transition in list(itr.product([0,1],[0,1],[0,1])):
            
            transition_prob = compute_transition_probability_voter(measure, [i,j,k], transition)
            new_coordinates = np.array([i,j,k]) + side_length*np.array(transition)
            new_measure[tuple(new_coordinates)] = \
                new_measure[tuple(new_coordinates)] + \
                measure[i,j,k]*transition_prob 
    
    return smoothen(new_measure, int(np.log2(side_length)))

def simulate_nonlinear_dynamics_voter(initial_measure):
    mu_0 = initial_measure
    mu_current = mu_0

    consec_diff = []
    while True:
        mu_next = transition_voter(mu_current)
        consec_diff = consec_diff + [np.sqrt(((mu_next - mu_current)**2).sum())] 
        mu_current = mu_next
        if consec_diff[-1] < 1e-10:
            break

    return (consec_diff, mu_current)

def simulate_nonlinear_dynamics_for_fixed_steps_voter(initial_measure,steps):
    mu_0 = initial_measure
    mu_current = mu_0

    consec_diff = []
    for i in range(steps):
        mu_next = transition_voter(mu_current)
        consec_diff = consec_diff + [np.sqrt(((mu_next - mu_current)**2).sum())] 
        mu_current = mu_next

    return (consec_diff, mu_current)

def l2norm(x,y):
    return np.sqrt(((x-y)**2).sum())

In [71]:
simulate_nonlinear_dynamics_for_fixed_steps_voter(create_random_k_measure(1),100)

([0.34027464177815092,
  0.23408228552094085,
  0.19109524292074084,
  0.16947881668279666,
  0.15809951896326152,
  0.14734576984210837,
  0.14186741507251902,
  0.13632645190779163,
  0.13288463995428779,
  0.12924386482279085,
  0.12696538666385529,
  0.12457931490263389,
  0.12281173012465467,
  0.12107514524542368,
  0.11973213417289444,
  0.11844658173449067,
  0.11732794103175598,
  0.11632574502946531,
  0.11541719103927907,
  0.11461817485156961,
  0.11383714525163298,
  0.11318275164750898,
  0.11252409257243555,
  0.11197641223531599,
  0.11139961040716767,
  0.11093470766183537,
  0.11043629300449495,
  0.11003412595889982,
  0.10959186461549718,
  0.10924140543523454,
  0.10885253527113478,
  0.10854225010376706,
  0.10819358224202395,
  0.1079178677285715,
  0.10760717835445349,
  0.10735893986183606,
  0.10707794243226047,
  0.10685408381072817,
  0.10660100458893236,
  0.10639692257671476,
  0.10616631766719732,
  0.10598017092804063,
  0.10577063386634669,
  0.10559930

In [68]:
measure.sum()

0.99999999999999989

# Extension to several dimensions